<img src="https://pbs.twimg.com/profile_images/928987025403006976/_1jMEtK9_400x400.jpg" alt="drawing" width="200px"/>

<h2 style="text-align: center; margin-top: .5em"> 
    [Universidade Federal de Campina Grande - UFCG](http://www.ufcg.edu.br/) 
</h2>
<h2 style="text-align: center; margin-top: .5em">  
    [Departamento de Sistemas e Computação](http://www.computacao.ufcg.edu.br/) 
</h2>

<div style="width: 35em; margin: auto; margin-top: 1em">
<h4 style="margin-top: .5em"> Disciplina: [Compiladores](http://www.dsc.ufcg.edu.br/~franklin/disciplinas/2018-1/Compiladores/) </h4>
<h4 style="margin-top: .5em"> Professor: [Leandro Balby Marinho](https://scholar.google.com.br/citations?user=6XO2tOwAAAAJ&hl=EN) </h4>
<h5 style="font-style: normal; margin-top: .5em"> Aluno: </h5>
<ul style="list-style-type: cjk-ideographic !important;"> <li>[Paulo Sérgio]( mailto:paulo.araujo@ccc.ufcg.edu.br ) </li></ul>
</div>
---

##### Fases da construção de um compilador 
Para construirmos um compilador que traduza código GO para Assembly temos que atender as etapas clássicas na construção de um compilador, sendo estas: 
    1. Análise Léxica 
    2. Análise Sintática
    3. Análise Semântica 
    
### 1. Análise Léxica

Para fazer a análise léxica usaremos a ferramenta [JFLEX](http://jflex.de), que é um analisador léxico escrito em java. O JFLEX utiliza um arquivo de entrada que especifíca a estrutura da linguagem. Um manual sobre JFLEX e como entregar com o CUP pode ser encontrado no seguinte endereço http://jflex.de/manual.pdf

Dentro do aquivo .lex temos que ligar trechos do CUP que nos ajudarão a aplicar as ações semânticas destinadas a tradução do código. 
```java
import java_cup.runtime.*;
import cup.sym; 
%cup
%cupdebug
%line 
%column 

/*
  Declarações
  Código entre %{ e %}, ambos dos quais devem estar no inicio de uma linha
  será copiado letra a letra para dentro do arquivo fonte Lexer. 
  Aqui você irá declarar variáveis e funções que serão utilizadas nas ações de 'scaneamento'
  
*/
%{   
    /* Para criar um novo java_cup.runtime.Symbol com informação sobre
       o token atual, mas esse tipo de token não tem valor associado. 
    */
    private Symbol symbol(int type) {
        return new Symbol(type, yyline, yycolumn);
    }
    
    /* Também cria um new java_cup.runtime.Symbol Com informação  
       sobre o token atual, mas esse objeto tem um valor. */
    private Symbol symbol(int type, Object value) {
        return new Symbol(type, yyline, yycolumn, value);
    }
%}
%% 
   ...
   {Numero}+{IS}?                            { return symbol(sym.CONSTANT, new String(yytext())); }

```

Isso deveria gerar uma saída que vai será o parser da linguagem depois. 

Links auxiliares:

    https://johnidm.gitbooks.io/compiladores-para-humanos/content/part1/lexical-analysis.html

    https://johnidm.gitbooks.io/compiladores-para-humanos/content/part2/building-the-first-lexical-analyzer-with-JFlex.html 


##### Executando a Análise Léxica  

Precisamoo portanto executar os seguintes passos 

1. Escrever o arquivo .lex com as regras necessárias para a linguagem GO. 
2. Escrever o arquivo .cup com a gramática e as ações semânticas que irá gerar o código assembly equivalente. 
3. Escrever um Compilador/Tradutor que irá utilizar o Lexer, Parser para gerar um arquivo em assembly. 
    
    

###### Utils

In [1]:
from os import popen
def cmd(cmd): print(popen(cmd).read()) # Executa um comando e imprime a saida.
    

- ###### Especificações no arquivo .lex

As regras adicionadas no arquivo .lex estão disponíveis no próprio site da especificação da linguagem. 
- https://golang.org/ref/spec

No arquivo .lex iremos definir: 
    - Padrões dos tokens escritos em expressão regulares
    - Ações para 'informar' ao CUP quando acharmos tokens.

Como o nosso arquivo .lex já está ligado com o cup, não podemos compilar e mostrar vagarosamente os passos 
incrementais para a construção do nosso tradutor(compilador) GO - ASSEMBLY. 

Assim devemos basear esse documento nos seguintes comandos necessários para a execução do código. 

``` bash
jflex src\flex\rules.flex
java -cp %JFLEX_HOME%\lib\java-cup-11a.jar;. java_cup.Main < spec.cup  # gera parser.java, sym.java apartir da gramática da linguagem
javac -cp %JFLEX_HOME%\lib\java-cup-11a.jar;. Translator.java # Gera tradutor : new parser(new Lexer(new FileReader(argv[0])))
java -cp %JFLEX_HOME%\lib\java-cup-11a.jar;. Translator <sourcefile>
```

###### Comando 1

In [2]:
cmd('jflex src/flex/rules.flex') # Cria um Lexer das regras da análise léxica

Reading "src\flex\rules.flex"
Constructing NFA : 565 states in NFA
Converting NFA to DFA : 
...................................................................................................................................................................................................................................................
245 states before minimization, 185 states in minimized DFA
Old file "src\flex\Lexer.java" saved as "src\flex\Lexer.java~"
Writing code to "src\flex\Lexer.java"



###### Comando 2  | Gerar parser e sym.java 

In [3]:
from os import sep, listdir, chdir, getcwd

cmd('java -cp %JFLEX_HOME%{0}lib{0}java-cup-11a.jar;src{0}flex{0}. java_cup.Main -destdir src{0}flex < src{0}flex{0}spec.cup '.format(sep))
cmd('jflex src{0}flex{0}rules.flex'.format(sep))
for elem in ['parser.java', 'sym.java']:
    assert elem in listdir('src/flex')


Reading "src\flex\rules.flex"
Constructing NFA : 565 states in NFA
Converting NFA to DFA : 
...................................................................................................................................................................................................................................................
245 states before minimization, 185 states in minimized DFA
Old file "src\flex\Lexer.java" saved as "src\flex\Lexer.java~"
Writing code to "src\flex\Lexer.java"



###### Comando 3 | Compilador/Tradutor

In [4]:
#javac -cp %JFLEX_HOME%\lib\java-cup-11a.jar;.  # Gera tradutor : new parser(new Lexer(new FileReader(argv[0])))

retdir = getcwd()
print(retdir)
chdir('src{0}flex'.format(sep))
print(listdir())
cmd('javac -cp %JFLEX_HOME%{0}lib{0}java-cup-11a.jar;src{0}flex{0};. Translator.java'.format(sep))
chdir(retdir)

C:\Users\Paulo\academic\gocomp
['.gitignore', 'CUP$parser$actions.class', 'Lexer.class', 'Lexer.java', 'Lexer.java~', 'parser.class', 'parser.java', 'rules.flex', 'spec.cup', 'sym.class', 'sym.java', 'test.txt', 'Translator.class', 'Translator.java']



In [5]:
retdir = getcwd()
chdir('src{0}flex'.format(sep))
cmd('java -cp %JFLEX_HOME%{0}lib{0}java-cup-11a.jar;src{0}flex{0};. Translator test.txt'.format(sep))

chdir(retdir)

Found identifier: main
Found keyword: func
Found identifier: main
Found operator/punctuation: (
Found operator/punctuation: )
Found operator/punctuation: {
Found identifier: println
Found operator/punctuation: (
Found string literal: "ola"
Found operator/punctuation: )
Found operator/punctuation: }
"ola"
compilation_unit matched
cunitcode
result porra



In [6]:
'java -cp %JFLEX_HOME%{0}lib{0}java-cup-11a.jar;src{0}flex{0};. Translator test.txt'.format(sep)

'java -cp %JFLEX_HOME%\\lib\\java-cup-11a.jar;src\\flex\\;. Translator test.txt'